In [1]:
import pandas as pd

# Charger le dataset
df = pd.read_csv("../data/ventes_produits_sales.csv")

# Afficher les 5 premières lignes
df.head()

,Ventes,Produits,Dates,Catégories
0,"486,1€",Casque,2023-01-17,Electronique
1,"158,05€",Haltères,2023-04-02,Sport
2,"124,55€",Lego,2023-03-21,Jouets
3,NaN,Lampe,2023-05-24,Maison
4,"289,93€",Ballon,2023-03-02,Sport


In [2]:
df.shape

(10200, 4)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10200 entries, 0 to 10199
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Ventes      9691 non-null   object
 1   Produits    9691 non-null   object
 2   Dates       9686 non-null   object
 3   Catégories  9706 non-null   object
dtypes: object(4)
memory usage: 318.9+ KB


In [4]:
df.describe()

,Ventes,Produits,Dates,Catégories
count,9691,9691,9686,9706
unique,8665,20,672,6
top,417.22,Lego,2023-12-11,Jouets
freq,4,512,27,1943


La colonne 'Ventes' représente le montant vendu par produit. Elle permet de calculer le chiffre d'affaires par jour, mais aussi d'identifier les produits les plus rentables ainsi que les jours où les ventes se font le plus.
La colonne 'Produits' représente le produit vendu. Elle permet d'identifier les produits les plus vendus par catégorie.
La colonne 'Dates' représente la date à laquelle le produit a été vendu. Elle permet d'identifier les périodes où les produits sont le plus vendus.
La colonne 'Catégories' représente la catégorie du produit vendu. Elle permet d'identifier la catégorie de produits la plus rentable.

# Types d'analyses business possibles avec ces données:
1. Analyse Temporelle (Saisonnalité et Tendances)
En combinant Ventes et Dates, vous déterminez la santé de l'activité dans le temps :
Croissance périodique : Évolution du chiffre d'affaires (CA) par mois ou par trimestre.
Saisonnalité : Identification des pics de vente (ex: soldes de janvier 2026, fêtes de fin d'année).
Ventes journalières : Déterminer les jours de la semaine les plus rentables pour optimiser les effectifs.
2. Analyse de Portefeuille (Performance Produits)
En combinant Ventes, Produits et Catégories :
Top/Flop 10 : Identification des produits "stars" qui génèrent le plus de CA et des produits "morts" à supprimer du catalogue.
Part de marché interne : Quelle Catégorie domine les ventes ? (ex: l'électronique représente-t-elle 60% du CA total ?).
Concentration (Loi de Pareto) : Est-ce que 20% de vos produits réalisent 80% de votre chiffre d'affaires ? [1, 3]
3. Analyse de Segmentation (Stratégie de Gamme)
En utilisant les Catégories :
Analyse de marge théorique : Si vous connaissez les marges par catégorie, vous pouvez diriger vos efforts marketing vers les catégories les plus rentables.
Diversification : Vérifier si une catégorie est trop dépendante d'un seul produit.

In [5]:
df.isnull().sum()

Ventes        509
Produits      509
Dates         514
Catégories    494
dtype: int64

In [6]:
df.dtypes

Ventes        object
Produits      object
Dates         object
Catégories    object
dtype: object

In [7]:
df.duplicated().sum()

np.int64(213)

In [10]:
# Suppression des doublons
df = df.drop_duplicates()

In [13]:
# Correction des types
# 1. Dates
df['Dates'] = pd.to_datetime(df['Dates'], dayfirst=True, format='mixed')
# 2. Ventes
df['Ventes'] = df['Ventes'].str.replace(',', '.').str.replace('€', '').str.strip()
df['Ventes'] = pd.to_numeric(df['Ventes'], errors='coerce')

In [14]:
# Gestion des valeurs manquantes
mediane_ventes = df['Ventes'].median()
df['Ventes'] = df['Ventes'].fillna(mediane_ventes)

In [15]:
cols_a_nettoyer = ['Produits', 'Catégories']

for col in cols_a_nettoyer:
    #Uniformisation des colonnes
    df[col] = df[col].str.capitalize()
    # Calcul du mode pour la colonne actuelle
    mode_val = df[col].mode()[0]
    # Remplissage des valeurs manquantes
    df[col] = df[col].fillna(mode_val)


In [16]:
df= df.dropna()

In [17]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 9495 entries, 0 to 10173
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Ventes      9495 non-null   float64       
 1   Produits    9495 non-null   object        
 2   Dates       9495 non-null   datetime64[ns]
 3   Catégories  9495 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 370.9+ KB


,Ventes,Dates
count,9495.000000,9495
mean,255.078161,2023-12-28 04:47:05.402843392
min,10.060000,2023-01-01 00:00:00
25%,139.245000,2023-07-02 00:00:00
50%,255.695000,2023-12-27 00:00:00
75%,370.090000,2024-06-23 00:00:00
max,499.910000,2024-12-28 00:00:00
std,138.103530,NaN


Dans la colonne Ventes, les ventes étaient des chaînes de caractères au lieu de nombres. Il a d'abord fallu les convertir en nombre pour les calculs ultérieurs en retirant le symbole € et en remplacant la virgule par un point. Puis les valeurs manquantes ont été remplacées par leur moyenne pour  conserver la cohérence des analyses.
Dans la colonne 'Dates', les dates étaient des chaînes de caractères. Leur type a été remplacé en 'datetime' afin de faciliter les calculs ultérieurs en ayant par exemple besoin d'un mois spécifique.
Dans la colonne 'Catégories', certaines catégories avaient des problèmes de casse, 'Sport' et 'sport' étaient deux catégories différentes ce qui n'est pas normalement le cas. Cela a été évité par le remplacement de chaque première lettre d'une catégorie par sa majuscule.

(IA)
1) Problèmes identifiés
Données manquantes (NaN) : Plus de 3 % de données manquantes par colonne. Cela fausse les calculs statistiques et empêche les analyses globales.
Types de données incorrects ("Object") : C’est le problème le plus critique.
Ventes : Doit être numérique (float) pour permettre des sommes ou moyennes.
Dates : Doit être au format datetime pour extraire les mois, années ou saisonnalités.
Doublons (213 lignes) : Ils gonflent artificiellement le chiffre d'affaires et faussent la réalité du business.
Incohérence potentielle : Le type "Object" cache souvent des erreurs de saisie (ex: "100€" au lieu de "100", ou des formats de date mixtes).
2) Actions à effectuer
Suppression des doublons : Utiliser df.drop_duplicates().
Conversion des types :
Transformer Dates avec pd.to_datetime().
Transformer Ventes avec pd.to_numeric(), en gérant les symboles monétaires si nécessaire.
Traitement des valeurs manquantes :
Ventes : Remplacer par la médiane (plus robuste que la moyenne) ou supprimer les lignes si la vente est l'information cruciale.
Catégories/Produits : Remplacer par une valeur par défaut comme "Inconnu" ou utiliser la valeur la plus fréquente (mode).
Dates : Si la date est manquante, la ligne est souvent inutilisable pour une analyse temporelle (suppression recommandée).
Normalisation : Mettre les Produits et Catégories en minuscules pour éviter que "Pain" et "pain" soient comptés séparément.
3) Pourquoi ces choix ?
Fiabilité (Doublons) : On ne peut pas prendre de décisions sur des données répétées par erreur technique.
Performance et Fonctionnalités (Types) : On ne peut pas tracer de graphique temporel si la colonne Dates est considérée comme du texte. De même, on ne peut pas additionner du texte.
Intégrité (Valeurs manquantes) :
Supprimer 300 lignes sur 10 200 (environ 3 %) est acceptable si cela garantit la précision.
Cependant, imputer (remplacer) par la médiane permet de conserver le volume de données sans trop biaiser les statistiques.
Standardisation : Cela garantit que les regroupements (groupby) par produit ou catégorie soient exacts et ne créent pas de catégories fantômes à cause d'une majuscule.

En statistiques, on considère souvent que la quasi-totalité des données (environ 95% dans une distribution normale) se situe à moins de deux écarts-types de la moyenne.
Toute donnée se trouvant à 3 ou 4 écarts-types de la moyenne est immédiatement identifiée comme une anomalie (une erreur de saisie, une fraude ou un événement exceptionnel).

In [18]:
df.to_csv("../data/sales_clean.csv", index=False)